In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import sklearn.gaussian_process.kernels as kernels

from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.decomposition import PCA
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.feature_selection import VarianceThreshold
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR

In [84]:
BASE_PATH = Path("/workspaces/age_from_brain_image_data/src/age_from_brain_image_data/data")
x_train = pd.read_csv(BASE_PATH / 'X_train.csv', skiprows=1, header=None).values[:, 1:]
x_test = pd.read_csv(BASE_PATH / 'X_test.csv', skiprows=1, header=None).values[:, 1:]
y_train = pd.read_csv(BASE_PATH / 'y_train.csv', skiprows=1, header=None).values[:, 1:].flatten()

In [85]:
def remove_outliers(x_train, y_train, contamination=0.047, random_state=42):

    x_train_df = pd.DataFrame(x_train).copy()
    y_train_series = pd.Series(np.asarray(y_train).flatten(), index=x_train_df.index)
    
    med = x_train_df.median(axis=0)
    xtr_imp = x_train_df.fillna(med)
    
    scaler = StandardScaler(with_mean=True, with_std=True)
    xtr_std = scaler.fit_transform(xtr_imp)
    
    pca = PCA(n_components=2, random_state=random_state)
    x_proj = pca.fit_transform(xtr_std)
    
    
    iso = IsolationForest(contamination=contamination, random_state=random_state)
    mask = iso.fit_predict(x_proj) == 1
    
    n_outliers = np.sum(~mask)
    print(f"[OutlierRemoval] Removed {n_outliers} outliers")
    
    if isinstance(x_train, np.ndarray):
        return x_train[mask], y_train[mask]
    else:
        return x_train_df[mask], y_train_series[mask]

In [86]:
class SpearmanSelector(BaseEstimator, TransformerMixin):
    """Select top K features by Spearman correlation with target."""
    
    def __init__(self, top_k=200):
        self.top_k = top_k
    
    def fit(self, x, y):
        if y is None:
            raise ValueError("SpearmanSelector requires y")
        
        x_df = pd.DataFrame(x) if not isinstance(x, pd.DataFrame) else x
        y_series = pd.Series(np.asarray(y).flatten(), index=x_df.index)
        
        spearman_corr = x_df.corrwith(y_series, method='spearman').abs()
        
        n_keep = min(self.top_k, len(spearman_corr))
        top_features = spearman_corr.nlargest(n_keep).index
        
        self.selected_features_ = top_features.tolist()
        self.n_features_in_ = x_df.shape[1]
        
        print(f"[SpearmanSelector] Selected {len(self.selected_features_)} features (from {self.n_features_in_})")
        
        return self
    
    def transform(self, X):
        x_df = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X
        return x_df.iloc[:, self.selected_features_].values


class RFSelector(BaseEstimator, TransformerMixin):
    """Select top K features by RandomForest feature importance."""
    
    def __init__(self, top_k=200, n_estimators=1000, max_depth=None, 
                 min_samples_leaf=1, max_features='sqrt', random_state=42):
        self.top_k = top_k
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.random_state = random_state
    
    def fit(self, x, y):
        
        if y is None:
            raise ValueError("RFSelector requires y")
        
        x_df = pd.DataFrame(x) if not isinstance(x, pd.DataFrame) else x
        y_series = pd.Series(np.asarray(y).flatten())
        
        self.rf_ = RandomForestRegressor(
            n_estimators=self.n_estimators,
            max_depth=self.max_depth,
            min_samples_leaf=self.min_samples_leaf,
            max_features=self.max_features,
            random_state=self.random_state,
            n_jobs=-1,
            verbose=0
        )
        self.rf_.fit(x_df, y_series)
        
        importances = pd.Series(self.rf_.feature_importances_, index=x_df.columns)
        importances = importances.sort_values(ascending=False)
        
        n_keep = min(self.top_k, len(importances)) if self.top_k else len(importances)
        self.selected_features_ = importances.head(n_keep).index.tolist()
        self.n_features_in_ = x_df.shape[1]
        
        print(f"[RFSelector] Selected {len(self.selected_features_)} features (from {self.n_features_in_})")
        
        return self
    
    def transform(self, x):
        x_df = pd.DataFrame(x) if not isinstance(x, pd.DataFrame) else x
        return x_df.iloc[:, self.selected_features_].values


In [87]:
class NaNAwareScaler(BaseEstimator, TransformerMixin):
    """
    Scaler that preserves NaN positions.
    1. Remember NaN positions
    2. Fill NaNs with median temporarily
    3. Scale the data
    4. Restore NaNs at original positions
    """

    def __init__(self, with_mean=True, with_std=True):
        self.with_mean = with_mean
        self.with_std = with_std

    def fit(self, x, y=None):
        x_df = pd.DataFrame(x)

        self.median_ = x_df.median(axis=0)

        x_filled = x_df.fillna(self.median_)

        self.scaler_ = StandardScaler(with_mean=self.with_mean, with_std=self.with_std)
        self.scaler_.fit(x_filled)

        return self

    def transform(self, x):
        x_df = pd.DataFrame(x)

        nan_mask = x_df.isna()

        x_filled = x_df.fillna(self.median_)

        x_scaled = self.scaler_.transform(x_filled)
        x_scaled_df = pd.DataFrame(x_scaled, index=x_df.index, columns=x_df.columns)

        x_scaled_df[nan_mask] = np.nan

        return x_scaled_df.values

In [88]:
def build_feature_pipeline(random_state=42):
    pipeline = Pipeline(
        [
            ("variance", VarianceThreshold()),
            ("scaler", NaNAwareScaler()),
            ("knn_imputer", KNNImputer()),
            ("spearman", SpearmanSelector()),
            ("rf_selector", RFSelector(random_state=random_state)),
        ]
    )

    return pipeline

In [89]:
def build_model(random_state=42):
    
    kernel = kernels.ConstantKernel(1.0, (1e-3, 1e3)) \
        * kernels.RationalQuadratic(length_scale=1.0, alpha=1.0) \
        + kernels.WhiteKernel(noise_level=1e-3, noise_level_bounds=(1e-6, 1e1))
    
    gpr = GaussianProcessRegressor(
        kernel=kernel,
        random_state=random_state
    )
    
    model = StackingRegressor(
        estimators=[
            ("svr", NuSVR()),
            ("hgb", HistGradientBoostingRegressor(random_state=random_state)),
            ("gp", gpr)
        ],
        final_estimator=LinearRegression()
    )
    
    return model

In [90]:
x_train_clean, y_train_clean = remove_outliers(
    x_train, y_train, contamination=0.047, random_state=42
)

[OutlierRemoval] Removed 57 outliers


In [ ]:
feature_pipeline = build_feature_pipeline()
model = build_model()

full_pipeline = Pipeline(
    [
        ("features", feature_pipeline),
        ("model", model),
    ]
)

param_grid = {
    "features__variance__threshold": [1e-7],
    "features__scaler__with_mean": [True],
    "features__scaler__with_std": [True],
    "features__knn_imputer__n_neighbors": [1],
    "features__knn_imputer__weights": ["distance"],
    "features__spearman__top_k": [202],
    "features__rf_selector__top_k": [173],
    "features__rf_selector__n_estimators": [1000],
    "features__rf_selector__max_depth": [None],
    "features__rf_selector__min_samples_leaf": [1],
    "features__rf_selector__max_features": ["sqrt"],

    # Model params - NuSVR
    "model__svr__C": [55],
    "model__svr__gamma": ["scale"],
    "model__svr__nu": [0.5],
    "model__svr__kernel": ["rbf"],
    
    # Model params - HistGradientBoosting
    "model__hgb__learning_rate": [0.1],
    "model__hgb__max_iter": [100],
    "model__hgb__max_depth": [None],
    "model__hgb__min_samples_leaf": [20],
    
    # Model params - GaussianProcess
    "model__gp__alpha": [0.0],
    "model__gp__normalize_y": [True],
    "model__gp__n_restarts_optimizer": [8],
}

grid = GridSearchCV(
    full_pipeline,
    param_grid,
    cv=3,
    scoring='r2',
    n_jobs=-1,
    verbose=3,
)

grid.fit(x_train_clean, y_train_clean)

y_test_pred = grid.predict(x_test)
print(f"\nTest predictions: shape={y_test_pred.shape}, mean={y_test_pred.mean():.2f}")

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[SpearmanSelector] Selected 202 features (from 828)
[SpearmanSelector] Selected 202 features (from 828)
[SpearmanSelector] Selected 202 features (from 828)
[RFSelector] Selected 173 features (from 202)
[RFSelector] Selected 173 features (from 202)
[RFSelector] Selected 173 features (from 202)


/workspaces/age_from_brain_image_data/.venv/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-06. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/workspaces/age_from_brain_image_data/.venv/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-06. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/workspaces/age_from_brain_image_data/.venv/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-06. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/

[SpearmanSelector] Selected 202 features (from 828)
[RFSelector] Selected 173 features (from 202)


/workspaces/age_from_brain_image_data/.venv/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-06. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/workspaces/age_from_brain_image_data/.venv/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-06. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/workspaces/age_from_brain_image_data/.venv/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-06. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/


Test predictions: shape=(776,), mean=69.88


In [ ]:
print(f"Best CV Score: {grid.best_score_:.4f}")
print(f"Best RMSE: {np.sqrt(-grid.best_score_):.4f}")

Best CV Score (neg MSE): -26.5114
Best RMSE: 5.1489
